In [1]:
from tqdm import tqdm
from pprint import pprint
import json
from sklearn.cross_validation import train_test_split
from random import random
import numpy as np
import _pickle as pkl
from collections import Counter

/home/ajinkya/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
with open('../data/train-v1.1.json', 'r') as f:
    train_data = json.load(f)
with open('../data/dev-v1.1.json', 'r') as f:
    test_data = json.load(f) # use as test

In [3]:
data_struct = {'train_X': [], 'train_y': [], 'test_X': [], 'test_y': []} # structured data
split_ratio = 0.2
data_tokens = []

for dataset in ['train', 'test']:
    if dataset == 'train':
        data = train_data
    else:
        data = test_data
    for d in tqdm(data['data']):
        paras = d['paragraphs']
        for para in paras:
            context = para['context']
            data_tokens.extend(context.lower().strip().split(" "))
            qas = para['qas']
            for qa in qas:
                q_text = qa['question']
                data_tokens.extend(q_text.lower().strip().split(" "))
                for a in qa['answers']:
                    start, ans_text = a['answer_start'], a['text']
                    X_q = (context, q_text, ans_text)
                    ans = (start, start+len(ans_text))
                    data_tokens.extend(ans_text.lower().strip().split(" "))
                    data_struct[dataset+'_X'].append(X_q)
                    data_struct[dataset+'_y'].append(ans)

def clean(token):
    cleaned_token = token.strip(".,?!-:;'()[]\"`")
    if cleaned_token[-2:] == "'s":
        cleaned_token = cleaned_token[:-2]
    if cleaned_token[-2:] == "'t":
        cleaned_token = cleaned_token[:-2]+'t'
    return cleaned_token

data_tokens = [clean(i.lower()) for i in data_tokens]
data_tokens = [i for i in data_tokens if len(i)>0]
data_vocab = Counter(data_tokens)

100%|██████████| 48/48 [00:00<00:00, 267.28it/s]


In [4]:
raw_data = {'X_train': None,
        'y_train': None,
        'X_val': None,
        'y_val': None,
        'X_test': data_struct['test_X'],
        'y_test': data_struct['test_y']}
raw_data['X_train'], raw_data['X_val'], raw_data['y_train'], raw_data['y_val'] \
     = train_test_split(data_struct['train_X'], data_struct['train_y'], test_size=0.25)
for k, v in raw_data.items():
    print(k, len(v))
with open('../data/data.json', 'w') as f:
    json.dump(raw_data, f)

X_test 34726
X_val 21900
y_test 34726
X_train 65699
y_train 65699
y_val 21900


In [ ]:
with open('../data/glove/glove.6B.50d.pkl', 'rb') as f:
    glove_dict = pkl.load(f)
print(len(glove_dict))

In [ ]:
oov = []
for w, c in data_vocab.most_common():
    if w not in glove_dict:
        oov.append((w, c))
pprint(sorted(oov, key=lambda x:-x[1])[:10])